In [173]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from keras.models import load_model

batch_size = 64
epochs = 100
latent_dim = 256 #latent dimentionality of the encoding space
num_samples = 10000

data_path = 'fra.txt'

In [31]:
import gradio as gr

In [73]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
#Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    #We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [159]:
input_texts[1500]

'Start now.'

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

In [4]:
print('Number of Samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max Sequence length for inputs:', max_encoder_seq_length)
print('Max Sequence Length for outputs:', max_decoder_seq_length)

Number of Samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max Sequence length for inputs: 14
Max Sequence Length for outputs: 59


In [5]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)])

In [6]:
# Now we will encode our data as one hot representation using basic numpy by initializing with zero
# We will encode our data acc. to the dimensions used below
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t>0:
            #decoder_target_data will be ahead by one timestep and will not include the start character
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [166]:
encoder_input_data[[0]].shape

(1, 14, 70)

In [9]:
#Define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#we discard encoder output and only keep the states
encoder_states = [state_h, state_c]

In [10]:
#set up the decoder, using 'encoder_states' as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
#We set up our decoder to return full output sequences, and to return internal states as well. We dont use the return states in the training but we will use them in inferece
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [12]:
# #this model will turn encoder_input_data and decoder_input_data into decoder_target_data
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#          batch_size=batch_size,
#          epochs=epochs,
#          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 13s 40ms/step - loss: 1.1346 - accuracy: 0.7382 - val_loss: 0.9912 - val_accuracy: 0.7210
Epoch 2/100
125/125 [==============================] - 3s 28ms/step - loss: 0.8307 - accuracy: 0.7761 - val_loss: 0.8324 - val_accuracy: 0.7672
Epoch 3/100
125/125 [==============================] - 4s 29ms/step - loss: 0.6659 - accuracy: 0.8139 - val_loss: 0.6986 - val_accuracy: 0.7983
Epoch 4/100
125/125 [==============================] - 3s 28ms/step - loss: 0.5770 - accuracy: 0.8329 - val_loss: 0.6302 - val_accuracy: 0.8164
Epoch 5/100
125/125 [==============================] - 3s 28ms/step - loss: 0.5250 - accuracy: 0.8468 - val_loss: 0.5777 - val_accuracy: 0.8310
Epoch 6/100
125/125 [==============================] - 3s 27ms/step - loss: 0.4896 - accuracy: 0.8569 - val_loss: 0.5526 - val_accuracy: 0.8396
Epoch 7/100
125/125 [==============================] - 3s 25ms/step - loss: 0.4621 - accuracy: 0.8643 - val_loss: 0.5265 - val_accuracy

125/125 [==============================] - 4s 28ms/step - loss: 0.0898 - accuracy: 0.9715 - val_loss: 0.5481 - val_accuracy: 0.8807
Epoch 58/100
125/125 [==============================] - 3s 27ms/step - loss: 0.0878 - accuracy: 0.9721 - val_loss: 0.5534 - val_accuracy: 0.8799
Epoch 59/100
125/125 [==============================] - 4s 28ms/step - loss: 0.0860 - accuracy: 0.9727 - val_loss: 0.5583 - val_accuracy: 0.8799
Epoch 60/100
125/125 [==============================] - 3s 28ms/step - loss: 0.0840 - accuracy: 0.9731 - val_loss: 0.5655 - val_accuracy: 0.8792
Epoch 61/100
125/125 [==============================] - 4s 29ms/step - loss: 0.0819 - accuracy: 0.9739 - val_loss: 0.5708 - val_accuracy: 0.8788
Epoch 62/100
125/125 [==============================] - 4s 29ms/step - loss: 0.0803 - accuracy: 0.9739 - val_loss: 0.5742 - val_accuracy: 0.8786
Epoch 63/100
125/125 [==============================] - 4s 29ms/step - loss: 0.0785 - accuracy: 0.9745 - val_loss: 0.5733 - val_accuracy: 0.879

In [167]:
model.save("s2s_model.keras")

In [174]:
model = load_model("s2s_model.keras")

In [15]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [30]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[[seq_index]]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !



In [142]:
input_seq = encoder_input_data[[1500]]
decoded_sentence = decode_sequence(input_seq)
print("Input sentence:",input_texts[1500])
print("Decoded sentence:", decoded_sentence)

Input sentence: Start now.
Decoded sentence: Commence maintenant.



In [182]:
encoder_input_data[[1500]].shape

(1, 14, 70)

In [221]:
sentence = ["I like rice."]

def encode_sent(sentence):
    encoder_input_data_1 = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')

    for input_text in sentence:
        for t, char in enumerate(input_text):
            encoder_input_data_1[0, t, input_token_index[char]] = 1.
        encoder_input_data_1[0, t + 1:, input_token_index[' ']] = 1.
    return encoder_input_data_1

In [222]:
encode_sent(sentence).shape

(1, 14, 70)

In [223]:
input_seqe = encode_sent(sentence)
decoded_sentence = decode_sequence(input_seqe)
print("Input sentence:" + sentence[0])
print("Decoded sentence:", decoded_sentence)

Input sentence:I like rice.
Decoded sentence: J'aime le riz.



In [ ]:
def translate(sentence):
    return "Hello " + name + "!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch()

In [144]:
sent = encoder_input_data[[1500]]
states_value = encoder_model.predict(sent, verbose=0)
states_value

# target_seq = np.zeros((1, 1, num_decoder_tokens))
# # Populate the first character of target sequence with the start character.
# target_seq[0, 0, target_token_index["\t"]] = 1.0
# target_seq

# # Sampling loop for a batch of sequences
# # (to simplify, here we assume a batch of size 1).

# stop_condition = False
# decoded_sentence = ""
# while not stop_condition:
#     output_tokens, h, c = decoder_model.predict(
#         [target_seq] + states_value, verbose=0
#     )

# output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
# output_tokens

#     # Sample a token
#     sampled_token_index = np.argmax(output_tokens[0, -1, :])
#     sampled_char = reverse_target_char_index[sampled_token_index]
#     decoded_sentence += sampled_char

#     # Exit condition: either hit max length
#     # or find stop character.
#     if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#         stop_condition = True

#     # Update the target sequence (of length 1).
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     target_seq[0, 0, sampled_token_index] = 1.0

#     # Update states
#     states_value = [h, c]

[array([[ 6.74238503e-02,  3.25155139e-01, -3.87952596e-01,
         -2.26639166e-01,  7.19183922e-01, -3.40588897e-01,
         -6.45807385e-01,  1.02819383e-01, -4.95427966e-01,
          6.83908641e-01, -6.89500989e-03, -9.06098559e-02,
         -1.15648541e-03, -2.49876991e-01, -3.05770934e-02,
         -2.92362303e-01,  2.77374238e-02, -5.56047782e-02,
         -9.17485595e-01, -5.20794690e-01, -7.13167572e-03,
         -7.13266879e-02, -8.24670672e-01,  4.48758975e-02,
         -7.40211010e-01, -3.88962738e-02,  5.54826319e-01,
         -1.21769600e-01,  1.04761302e-01, -3.99822555e-03,
         -9.68667448e-01,  6.72630131e-01,  1.96202248e-02,
         -9.03229937e-02, -3.07985172e-02,  8.36903691e-01,
          8.70826747e-03,  7.98140988e-02,  7.90095806e-01,
          1.20249711e-01, -7.12742507e-01, -1.37288183e-01,
          2.89522205e-02,  8.20838809e-02, -4.22810316e-01,
          2.37524554e-01, -9.78699267e-01, -5.45916334e-03,
         -7.71316767e-01,  1.39725702e-0

In [147]:
sent = "Start now."
states_value = encoder_model.predict(encode_sent(sent), verbose=0)
states_value

# target_seq = np.zeros((1, 1, num_decoder_tokens))
# # Populate the first character of target sequence with the start character.
# target_seq[0, 0, target_token_index["\t"]] = 1.0
# target_seq

# # Sampling loop for a batch of sequences
# # (to simplify, here we assume a batch of size 1).

# stop_condition = False
# decoded_sentence = ""
# while not stop_condition:
#     output_tokens, h, c = decoder_model.predict(
#         [target_seq] + states_value, verbose=0
#     )

# output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
# output_tokens

#     # Sample a token
#     sampled_token_index = np.argmax(output_tokens[0, -1, :])
#     sampled_char = reverse_target_char_index[sampled_token_index]
#     decoded_sentence += sampled_char

#     # Exit condition: either hit max length
#     # or find stop character.
#     if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#         stop_condition = True

#     # Update the target sequence (of length 1).
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     target_seq[0, 0, sampled_token_index] = 1.0

#     # Update states
#     states_value = [h, c]

[array([[ 6.83594167e-01,  7.09279358e-01, -7.30389476e-01,
         -2.45467201e-02,  6.81126595e-01, -1.26257911e-02,
         -3.42536688e-01,  4.54822719e-01,  1.51101887e-01,
          8.05185258e-01, -9.94192995e-03, -1.96738709e-02,
         -6.99171273e-04, -8.73831660e-02, -8.41585221e-04,
         -4.48560864e-02, -7.74175107e-01, -1.24730198e-02,
         -9.89764929e-01,  9.12701666e-01, -3.85002815e-03,
         -1.25916833e-02, -5.65486312e-01, -1.85851417e-02,
         -2.46974200e-01, -1.46131692e-02,  9.02582824e-01,
          8.58879834e-02,  3.81625026e-01, -2.57128552e-02,
         -8.69388998e-01,  2.06898272e-01,  3.72548006e-03,
         -8.28406751e-01, -3.56795406e-03,  8.12657714e-01,
          2.20947084e-03,  7.67816678e-02,  7.60509610e-01,
         -1.28583848e-01,  8.65153134e-01,  4.53015864e-01,
          3.75953200e-03,  1.39377303e-02, -7.87973523e-01,
          1.60999492e-01, -8.47858965e-01, -2.28355266e-03,
         -4.91926074e-01,  1.24881849e-0

In [152]:
encoder_input_data[[1500]].shape

(1, 14, 70)

In [155]:
encode_sent("Start now").shape

(2, 14, 70)